# Soft Margin SVM

Hard Margin SVM 的缺點是可能 overfit, 因為 轉換項次太高 和 堅持所有資料都要區分開。

參考在 pocket 中的做法，最小化犯錯的數量:

$$
\min_{b,w} \sum_{n=1}^N \Big[ y_n \ne sign\big( w^T z_n + b \big) \Big]_{boolean}
$$

原來的 hard-margin SVM 問題是這樣定義:

$
\min_{b,w} \frac{1}{2} w^T w \\
s.t. \ \ y_n \big( w^T z_n + b \big) \ge 1 \text{ for all n }
$

結合 pocket 的方式，將 hard-margin SVM 修改成最小化犯錯的數量:

$
\min_{b,w} \frac{1}{2} w^T w + C \cdot \sum_{n=1}^N \Big[ y_n \ne sign\big( w^T z_n + b \big) \Big]_{boolean} \\
s.t. \ \ y_n \big( w^T z_n + b \big) \ge 1 \text{ for correct n } \\
s.t. \ \ y_n \big( w^T z_n + b \big) \ge -\infty \text{ for incorrect n }
$

C: trade-off of LARGE MARGIN and NOISE TOLERANCE.

C 越大 : 傾向不要犯錯；  
C 越小 : 傾向找到大的 margin；在對的點上要有寬的 margin。

將兩個 s.t. such that 的條件結合起來變成:

$
s.t. \ \ y_n \big( w^T z_n + b \big) \ge 1 - \infty \cdot \Big[ y_n \ne sign\big( w^T z_n + b \big) \Big]_{boolean} \\
$

表示 對的點時候一樣要在意 s.t. 條件；而錯的點時候要 大於 乘以負無限大，就和無視條件一樣。  
所以是對的點要限制 s.t. 條件；錯的點不限制 s.t. 條件。

boolean 的條件有個缺點，造成不再是 QP 問題；就用不上之前的 dual, kernel...

也無法分辨錯誤到底有多少；有時候可能錯誤很小，很接近邊界。

#### 使用錯誤衡量 margin violation : $ \xi $ - linear constraints

penalize with margin violation instead of error count - quadratic objective.

## Soft Margin SVM primal

<img src="./imgs/c204-margin-violation.png" style="float:right" />

$
\min_{b,w,\xi} \frac{1}{2} w^T w + C \cdot \sum_{n=1}^N \xi_n \\
s.t. \ \ y_n \big( w^T z_n + b \big) \ge 1 - \xi_n \text{ and } \xi_n \ge 0 \text{ for all n }
$

這樣就是一個 QP 問題:

objectives : $ w, \xi $ 的二次式 (實際上 $ \xi $ 只有一次)  
constraints : $ \xi $ 的一次式

### Lagrange Dual

將上面 primal 形式轉換成 Dual 的形式，Lagrange function with Lagrange multipliers $ \alpha_n $ and $ \beta_n $

$$
\begin{align}
\mathcal{L} \big( b, w, \xi, \alpha, \beta \big) = &
\frac{1}{2} w^T w + C \cdot \sum_{n=1}^N \xi_n \\
& + \sum_{n=1}^N \alpha_n \cdot \big( 1 - \xi_n - y_n ( w^T z_n + b) \big) +
\sum_{n=1}^N \beta_n \cdot \big( - \xi_n \big)
\end{align}
$$

獲得 Lagrange Dual 的形式，然後用 KKT condition 簡化該問題:

$$
\max_{\alpha_n \ge 0, \beta_n \ge 0} \Big( \min_{b,w,\xi} \mathcal{L} \big( b, w, \xi, \alpha, \beta \big) \Big)
$$

對 primal 裏頭每個變數的微分為 0:

$
\frac{\partial \mathcal{L}}{\partial \xi} = 0 = C - \alpha_n - \beta_n \iff \beta_n = C - \alpha_n
$

因為 $ \alpha_n \ge 0, \beta_n \ge 0 $, 所以 $ 0 \le \alpha_n \le C $

$$
\max_{0 \le \alpha_n \le C, \beta_n = C - \alpha_n} \Big( \min_{b,w,\xi} \ \ 
\frac{1}{2} w^T w
+ \sum_{n=1}^N \alpha_n \cdot \big( 1 - y_n ( w^T z_n + b) \big) \ \  \Big)
$$

inner problem same as hard-margin SVM, 條件稍有不同，加上 $ 0 \le \alpha_n \le C $

$
\frac{\partial \mathcal{L}}{\partial b} = 0 \ \ \iff \ \ \sum_{n=1}^N \alpha_n y_n = 0
$

$
\frac{\partial \mathcal{L}}{\partial w_i} = 0 \ \ \iff \ \ w = \sum_{n=1}^N \alpha_n y_n z_n
$


## Standard Soft-Margin SVM Dual

$$
\begin{align}
\min_{\alpha} & \frac{1}{2} \sum_{n=1}^{N} \sum_{m=1}^{N} \alpha_n \alpha_m y_n y_m z_n^T z_m - \sum_{n=1}^{N} \alpha_n \\
\text{ subject to } & \sum_{n=1}^{N} y_n \alpha_n = 0 \\
& 0 \le \alpha_n \le C, \text{ for n = 1,2,...,N } \\
\text{ implicity } & w = \sum_{n=1}^{N} y_n \alpha_n z_n = 0 \\
& \beta_n = C - \alpha_n, \text{ for n = 1,2,...,N }
\end{align}
$$

與原先的 Hard-Margin SVM 差別只在：用 C 作為 alpha 值上限設定。  
是一個 (convex) QP with N variables and 2N+1 constraints.

#### Soft Margin SVM - solve b - complementary slackness

$
\alpha_n \big( 1 - \xi_n - y_n ( w^T z_n + b ) \big) = 0 \\
\big( C - \alpha_n \big) \xi_n = 0 \ \ \to \ \ \beta_n ( \xi ) = 0
$

在 SV 上 $ SV ( \alpha_s \gt 0 ) $  
$ \to b = y_s - y_s \xi_s - w^T z_s $

在 free SV 上 $ (\alpha \le C) $  
$ \to \xi_s = 0 $

所以可以利用 free $ SV(x_s, y_x) $ 來獲得 b:

$$
b = y_s - \sum_{\text{SV indices n}} \alpha_n y_n K \big( x_n, x_s \big)
$$

## Kernel Soft-Margin SVM

$ q_{n,m} = y_n y_m K(x_n, x_m) $

$ \vec{p} = -1_{N} $

(A, c) for equ. / lower-bound / upper-bound constraints.

$ \alpha \leftarrow QP(Q_D, p, A, c) $

$ b = y_s - \sum_{\text{SV indices n}} \alpha_n y_n K \big( x_n, x_s \big) $

return SVs and their $ \alpha_n $ as well as b such that for new x,

$$
g_{SVM} \big ( x \big ) = 
sign \Big( \sum_{\text{SV indices n}} \alpha_n y_n K(x_n, x) + b \Big)
$$

<img src="./imgs/c204-free-svs.png" style="float:right;width:200px;" />

#### Physical meaning of $ \alpha_n $

Complementary Slackness:  
$
\alpha_n \big( 1 - \xi_n - y_n ( w^T z_n + b ) \big) = 0 \\
\big( C - \alpha_n \big) \xi_n = 0 \ \ \to \ \ \beta_n ( \xi ) = 0
$

- 右圖方塊: free SV $ 0 \le \alpha_n \le C, \xi_n = 0 $ 剛好在邊界上，可定位 b

- 右圖圓圈叉叉: Non SV, $ \alpha_n = 0, \xi_n = 0 $ 在邊界外，正確的分類；極少數可能在邊界上

- 右方三角: Bounded SV, $ \alpha_n = C $, 在邊界內，有違反 $ \xi_n \gt 0 $ (violation amount) 

### Model Selection

在 Soft Margin Gaussian Kernel 中，有兩個參數 $ C, \gamma $ 可以選擇。

利用 **Cross Validation** 來選擇好的 model.

### Leave-One-Out CV Error for SVM

recall: $ E_{LOOCV} = E_{CV} $ with N folds.

claim: $ E_{LOOCV} \le \frac{\#SV}{N} $

因為: non SV 的 $ e_{\text{non SV}} = 0, e_{SV} \le 1 $, 所以所有 N 相加後，可得上式。

所以 number of SVs 和 error 是有關係的，也可以用 # of SVs 來選擇好的 model.

- 可以用 # of SVs 來選擇好的 model (SV數量少的好)
- 只是 $ e_{LOOCV} $ 的 upper bound, 所以不可完全依賴；通常用來排除危險的 model (SV 太大)
- 在計算 $ e_{CV} $ 太麻煩時候，可以做為替代方案。
